In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import tensorflow_decision_forests as tfdf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-07-08 12:46:21.518517: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 12:46:21.518629: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 12:46:21.664388: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
train_df=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

Split training data for training and testing using train test split

In [3]:
from sklearn.model_selection import train_test_split
x=train_df.drop(columns=['SalePrice'])
y=train_df['SalePrice']
x_train, y_train, x_test, y_test=train_test_split(x,y,test_size=0.1, random_state=42)

Converted the training data from a dataframe to tensor flow data set to allow for Random Forest Modeling

In [4]:
train_tf=tfdf.keras.pd_dataframe_to_tf_dataset(train_df,label='SalePrice',task=tfdf.keras.Task.REGRESSION)
model=tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION)
model.fit(train_tf)

Use /tmp/tmpm4epmwl7 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:06.222704. Found 1460 examples.
Training model...


[INFO 24-07-08 12:46:43.3662 UTC kernel.cc:1233] Loading model from path /tmp/tmpm4epmwl7/model/ with prefix 16f1a7ed998943e3


Model trained in 0:00:02.829208
Compiling model...


[INFO 24-07-08 12:46:43.8650 UTC decision_forest.cc:660] Model loaded with 300 root(s), 138084 node(s), and 75 input feature(s).
[INFO 24-07-08 12:46:43.8651 UTC abstract_model.cc:1344] Engine "RandomForestOptPred" built
[INFO 24-07-08 12:46:43.8651 UTC kernel.cc:1061] Use fast generic engine


Model compiled.


In [5]:
test_df=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
test_df.shape

(1459, 80)

Highlighted the column ID in test_df 

In [6]:
test_id = test_df['Id']

Converted the test dataframe to tf dataset and used the RF model to predict the test tf data set

In [7]:
test_tf=tfdf.keras.pd_dataframe_to_tf_dataset(test_df)
pred=model.predict(test_tf)

2/2 [==============================] - 1s 28ms/step


converted the output back to dataframe and to csv

In [8]:
output = pd.DataFrame({'Id':test_id,'SalePrice': pred.squeeze()})
output.to_csv('output.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
